# Architecture Decision: Material File Format Mapping
## Direct ORMAE Format vs Intermediate Platform Format (SFTP)

**Context:**
- ORMAE optimization model consumes specific input file format
- Your platform uses SFTP for file transfer
- Need to decide: Generate ORMAE format directly OR intermediate format?

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("Architecture Decision Analysis")
print("="*100)

## OPTION A: Direct ORMAE Format Mapping

In [ ]:
option_a = {
    'name': 'Direct ORMAE Format',
    'description': 'Biocon Files → ORMAE Input Format directly (Multi-sheet Excel)',
    'architecture': [
        'Biocon DP Shortage → ORMAE BOM Sheet',
        'Biocon ParkourSC → ORMAE SKU/Demand Sheet',
        'Resource Mapping → ORMAE Resource Sheet',
        'Multi-sheet Excel → SFTP → ORMAE Model'
    ],
    'advantages': [
        'Single transformation: Biocon → ORMAE (no intermediate layer)',
        'Faster time-to-model',
        'Minimal data duplication',
        'Direct control of ORMAE schema',
        'Lower storage overhead (SFTP)',
        'Fewer transformation points = lower error risk'
    ],
    'disadvantages': [
        'ORMAE format tightly coupled to Biocon ETL',
        'Hard to support other solvers/tools later',
        'Changes in ORMAE schema = rework entire pipeline',
        'Difficult to debug/audit transformations',
        'No intermediate validation point',
        'Business logic mixed with ORMAE-specific logic'
    ],
    'complexity': 'Medium',
    'maintenance': 'Hard',
    'flexibility': 'Low',
    'scalability': 'Low'
}

print(f"\nOPTION A: {option_a['name']}")
print(f"Description: {option_a['description']}")
print(f"\nArchitecture Flow:")
for i, step in enumerate(option_a['architecture'], 1):
    print(f"  {i}. {step}")

print(f"\nAdvantages (+):")
for adv in option_a['advantages']:
    print(f"  ✓ {adv}")

print(f"\nDisadvantages (-):")
for dis in option_a['disadvantages']:
    print(f"  ✗ {dis}")

print(f"\nMetrics:")
print(f"  Complexity: {option_a['complexity']}")
print(f"  Maintenance: {option_a['maintenance']}")
print(f"  Flexibility: {option_a['flexibility']}")
print(f"  Scalability: {option_a['scalability']}")

## OPTION B: Intermediate Platform Format (Recommended)

In [ ]:
option_b = {
    'name': 'Intermediate Platform Format',
    'description': 'Biocon Files → Platform Standard Format → ORMAE Input Format',
    'architecture': [
        'Biocon DP Shortage → Platform BOM Standard',
        'Biocon ParkourSC → Platform Demand Standard',
        'Biocon Resources → Platform Resource Standard',
        'Platform Format (SFTP) → ORMAE Adapter → ORMAE Format',
        'Optional: Platform Format → Other Solvers (Gurobi, CPLEX, etc)'
    ],
    'advantages': [
        'Decouples Biocon ETL from solver-specific logic',
        'Platform format = source of truth (single validation point)',
        'Easy to add new solvers (ORMAE, Gurobi, CPLEX, custom)',
        'Solver changes don\'t affect Biocon pipeline',
        'Clear data quality gates (validate before adapting)',
        'Auditable transformation (intermediate artifact)',
        'Reusable for analytics, reporting, other use cases',
        'Easier to debug (check intermediate output)',
        'Future-proof (add new solvers without rework)',
        'Industry standard approach (ETL → Hub → Applications)'
    ],
    'disadvantages': [
        'Additional transformation layer',
        'Slightly longer development time',
        'More files stored on SFTP (BOM standard + ORMAE format)',
        'Need to define platform standard schema upfront',
        'Requires adapter maintenance for each solver'
    ],
    'complexity': 'Medium-High',
    'maintenance': 'Easy',
    'flexibility': 'High',
    'scalability': 'High'
}

print(f"\n\nOPTION B: {option_b['name']}")
print(f"Description: {option_b['description']}")
print(f"\nArchitecture Flow:")
for i, step in enumerate(option_b['architecture'], 1):
    print(f"  {i}. {step}")

print(f"\nAdvantages (+):")
for adv in option_b['advantages']:
    print(f"  ✓ {adv}")

print(f"\nDisadvantages (-):")
for dis in option_b['disadvantages']:
    print(f"  ✗ {dis}")

print(f"\nMetrics:")
print(f"  Complexity: {option_b['complexity']}")
print(f"  Maintenance: {option_b['maintenance']}")
print(f"  Flexibility: {option_b['flexibility']}")
print(f"  Scalability: {option_b['scalability']}")

## Comparison Matrix

In [ ]:
comparison = pd.DataFrame([
    {'Criteria': 'Setup Time', 'Direct ORMAE': '2-3 weeks', 'Intermediate Format': '4-5 weeks'},
    {'Criteria': 'Time-to-First-Model', 'Direct ORMAE': 'Fast', 'Intermediate Format': 'Medium'},
    {'Criteria': 'Adding New Solver', 'Direct ORMAE': 'Weeks', 'Intermediate Format': '1-2 days'},
    {'Criteria': 'Maintenance Effort', 'Direct ORMAE': 'High', 'Intermediate Format': 'Low'},
    {'Criteria': 'Debugging Complexity', 'Direct ORMAE': 'Hard', 'Intermediate Format': 'Easy'},
    {'Criteria': 'Data Quality Assurance', 'Direct ORMAE': 'End-to-end', 'Intermediate Format': 'Layer-by-layer'},
    {'Criteria': 'SFTP Storage', 'Direct ORMAE': 'Minimal', 'Intermediate Format': 'Moderate'},
    {'Criteria': 'Reusability', 'Direct ORMAE': 'Low', 'Intermediate Format': 'High'},
    {'Criteria': 'Future-Proofing', 'Direct ORMAE': 'Low', 'Intermediate Format': 'High'},
    {'Criteria': '5-Year Cost', 'Direct ORMAE': 'High', 'Intermediate Format': 'Low'}
])

print("\n\nCOMPARISON MATRIX:")
print(comparison.to_string(index=False))

## Recommended Architecture: Intermediate Platform Format

In [ ]:
print("\n" + "="*100)
print("RECOMMENDATION: OPTION B - INTERMEDIATE PLATFORM FORMAT")
print("="*100)

print("""
(Why? ORMAE is ONE solver. Your platform will evolve.)

📐 PROPOSED ARCHITECTURE:

┌─────────────────────────────────────────────────────────────────┐
│ LAYER 0: SOURCE DATA (Biocon Files)                             │
│                                                                  │
│ • DP Shortage (20251006-DP Material Shortage - Working file)    │
│ • ParkourSC_SNP (Demand, Resources, SKU info)                  │
│ • Resource Mapping Master (Resource allocation)                 │
│ • Filling, Assembly & Packing Mapping (Hierarchy)              │
└────────────────┬────────────────────────────────────────────────┘
                 │
                 │ ETL Process (Your current work)
                 ▼
┌─────────────────────────────────────────────────────────────────┐
│ LAYER 1: PLATFORM STANDARD FORMAT (Source of Truth)             │
│                                                                  │
│ Standard data model (NOT solver-specific)                       │
│ • BOM_Standard.xlsx (Material hierarchy, lead times)            │
│ • Demand_Standard.xlsx (SKU demand, planning data)              │
│ • Resource_Standard.xlsx (Production resources, capacities)    │
│ • Hierarchy_Standard.xlsx (7→8 series mappings)                 │
│                                                                  │
│ ✓ Single source of truth                                        │
│ ✓ Data quality validated once                                   │
│ ✓ Reusable by all tools/solvers                                │
│ ✓ SFTP stored for accessibility                                │
└────────────────┬────────────────────────────────────────────────┘
                 │
                 │ Solver-Specific Adapters (Lightweight)
                 ├─ ORMAE Adapter (Multi-sheet Excel)
                 ├─ Gurobi Adapter (CSV/LP format)
                 ├─ CPLEX Adapter (OPL format)
                 └─ Custom Solver Adapter (JSON/API)
                 ▼
┌─────────────────────────────────────────────────────────────────┐
│ LAYER 2: SOLVER INPUT FORMATS                                   │
│                                                                  │
│ • ORMAE_Input_YYYY-MM-DD.xlsx (Multi-sheet format)             │
│ • Gurobi_Input_YYYY-MM-DD.lp (LP format)                        │
│ • CPLEX_Input_YYYY-MM-DD.opl (OPL format)                       │
│ • Custom_Input_YYYY-MM-DD.json (API format)                     │
└─────────────────────────────────────────────────────────────────┘
""")

print("\n📊 BENEFITS OF THIS APPROACH:\n")

benefits = [
    ('Separation of Concerns', 'Biocon ETL ≠ Solver Logic'),
    ('Flexibility', 'Swap solvers without reworking ETL'),
    ('Quality Assurance', 'Validate platform format once, use everywhere'),
    ('Reusability', 'BOM data used for reporting, analytics, forecasting'),
    ('Maintainability', 'Changes isolated to adapter layer'),
    ('Scalability', 'Add new solvers with minimal rework'),
    ('Auditability', 'Clear intermediate artifacts for compliance'),
    ('Team Structure', 'ETL team ≠ Solver expert team')
]

for i, (benefit, detail) in enumerate(benefits, 1):
    print(f"  {i}. {benefit:25s} → {detail}")

## Implementation Plan

In [ ]:
print("\n" + "="*100)
print("IMPLEMENTATION PLAN: PLATFORM STANDARD FORMAT")
print("="*100)

implementation = {
    'Phase 1: Define Standards': [
        'Define BOM_Standard schema (Material, QTY, Lead Time, Resource, Section)',
        'Define Demand_Standard schema (SKU, Molecule, Product, Region, Demand_Q)',
        'Define Resource_Standard schema (Resource_ID, Capacity, Rate, Stage)',
        'Define Hierarchy_Standard schema (7-series, 8-series, Mapping, Validity)'
    ],
    'Phase 2: Create ETL': [
        'Biocon Files → BOM_Standard.xlsx',
        'Biocon Files → Demand_Standard.xlsx',
        'Biocon Files → Resource_Standard.xlsx',
        'Mapping File → Hierarchy_Standard.xlsx',
        'Validation: Check data quality, completeness'
    ],
    'Phase 3: ORMAE Adapter': [
        'BOM_Standard → ORMAE BOM_Sheet',
        'Demand_Standard → ORMAE Demand_Sheet',
        'Resource_Standard → ORMAE Resource_Sheet',
        'Hierarchy_Standard → ORMAE Mapping_Sheet'
    ],
    'Phase 4: Future Solvers': [
        'Create Gurobi Adapter (as needed)',
        'Create CPLEX Adapter (as needed)',
        'Create Custom Solver Adapter (as needed)'
    ]
}

for phase, tasks in implementation.items():
    print(f"\n{phase}:")
    for i, task in enumerate(tasks, 1):
        print(f"  {i}. {task}")

## File Structure (SFTP)

In [ ]:
print("\n" + "="*100)
print("PROPOSED SFTP FILE STRUCTURE")
print("="*100)

file_structure = """
/biocon_data/
├── input_sources/
│   ├── DP Shortage/
│   │   └── 20251006-DP Material Shortage - Working file.xlsx
│   ├── ParkourSC/
│   │   ├── ParkourSC_SNP.xlsx
│   │   └── Aug'25_L2_DP_Plan_Circulation_V2.xlsx
│   └── Mapping/
│       ├── Filling, Assembly & Packing Mapping.xlsx
│       └── Resource_Mapping_Master.xlsx
│
├── platform_standards/ (✓ SOURCE OF TRUTH - Validate once, reuse always)
│   ├── BOM_Standard_2025-01-15.xlsx
│   ├── Demand_Standard_2025-01-15.xlsx
│   ├── Resource_Standard_2025-01-15.xlsx
│   └── Hierarchy_Standard_2025-01-15.xlsx
│
├── solver_inputs/
│   ├── ormae/
│   │   └── ORMAE_Input_2025-01-15.xlsx (Multi-sheet)
│   ├── gurobi/ (Future)
│   │   └── Gurobi_Input_2025-01-15.lp
│   ├── cplex/ (Future)
│   │   └── CPLEX_Input_2025-01-15.opl
│   └── custom/ (Future)
│       └── Custom_Input_2025-01-15.json
│
├── validation_reports/
│   ├── Data_Quality_Report_2025-01-15.xlsx
│   ├── Completeness_Check_2025-01-15.xlsx
│   └── Gap_Analysis_2025-01-15.xlsx
│
└── outputs/
    ├── ormae_results/
    │   └── Optimization_Results_2025-01-15.xlsx
    └── analytics/
        ├── Demand_Analysis_2025-01-15.xlsx
        ├── Resource_Utilization_2025-01-15.xlsx
        └── BOM_Insights_2025-01-15.xlsx
"""

print(file_structure)

## Final Decision Framework

In [ ]:
print("\n" + "="*100)
print("DECISION FRAMEWORK: WHICH APPROACH?")
print("="*100)

decision_framework = """
Choose OPTION A (Direct ORMAE) IF:
  ✓ ORMAE is your ONLY optimization tool forever
  ✓ Timeline pressure is critical (need model running NOW)
  ✓ Small scope (single product, simple BOM)
  ✓ No future plans for analytics/reporting/other solvers

Choose OPTION B (Intermediate Platform Format) IF:
  ✓ Platform will grow beyond ORMAE
  ✓ May use multiple solvers (ORMAE, Gurobi, CPLEX)
  ✓ Need data for analytics, reporting, dashboards
  ✓ Teams are separate (ETL ≠ Solver experts)
  ✓ Long-term maintainability matters
  ✓ Data quality validation is critical
  ✓ Want to minimize rework if solver changes

───────────────────────────────────────────────────────────
🎯 FOR BIOCON/ORMAE PARTNERSHIP:
───────────────────────────────────────────────────────────

RECOMMENDATION: OPTION B (Intermediate Platform Format)

Why?
  1. ORMAE is one of many tools you might use
  2. Your SFTP platform suggests data hub architecture
  3. Biocon will likely use data for other purposes (reporting, forecasting)
  4. Separation of concerns = easier maintenance
  5. Industry best practice (cloud, API-first architectures follow this)
  6. Long-term cost savings outweigh initial setup overhead
"""

print(decision_framework)

## Next Actions

In [ ]:
print("\n" + "="*100)
print("NEXT ACTIONS")
print("="*100)

next_actions = """
1. CONFIRM DIRECTION with stakeholders:
   □ Will ORMAE be the only optimization tool?
   □ Will other teams need BOM data for analytics/reporting?
   □ Is long-term maintainability a priority?

2. IF OPTION B (Recommended):
   □ Define BOM_Standard schema (with ORMAE & Biocon teams)
   □ Define Demand_Standard schema
   □ Define Resource_Standard schema
   □ Define Hierarchy_Standard schema
   
3. CREATE ETL PIPELINE:
   □ Build Biocon Files → Platform Standards transformation
   □ Add data quality validation checks
   □ Create SFTP upload automation
   
4. BUILD ORMAE ADAPTER:
   □ Define ORMAE input format requirements
   □ Build Platform Standards → ORMAE Format transformation
   □ Create validation before sending to solver
   
5. DOCUMENTATION:
   □ Platform standard schemas (business dictionary)
   □ ETL pipeline documentation (technical)
   □ ORMAE adapter documentation (integration guide)
   □ Data governance (ownership, update frequency, validation)
"""

print(next_actions)

In [ ]:
print("\n" + "="*100)
print("SUMMARY")
print("="*100)

summary = """
┌─────────────────────────────────────────────────────────────────┐
│ RECOMMENDATION: Intermediate Platform Format (OPTION B)          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│ Biocon Files                                                     │
│     ↓                                                             │
│ ETL Pipeline (Your current work)                                 │
│     ↓                                                             │
│ Platform Standard Format ← (Source of Truth - SFTP stored)       │
│     ↓                                                             │
│ ORMAE Adapter (Lightweight transformation)                       │
│     ↓                                                             │
│ ORMAE Input Format (Ready for optimization model)                │
│                                                                  │
│ ─────────────────────────────────────────────────────────────   │
│ Benefits:                                                        │
│ ✓ Decouples Biocon ETL from ORMAE                              │
│ ✓ Data quality validated once at platform layer                │
│ ✓ Reusable by analytics, reporting, future solvers             │
│ ✓ Easy to maintain and extend                                   │
│ ✓ Industry standard architecture                                │
│ ─────────────────────────────────────────────────────────────   │
│ Timeline: 4-5 weeks vs 2-3 weeks                               │
│ But: Long-term cost/effort savings are significant             │
└─────────────────────────────────────────────────────────────────┘
"""

print(summary)